In [20]:
import requests

import pandas as pd
%pip install streamlit
%pip install altair pandas vega_datasets
%pip install -U altair_viewer
import altair as alt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [35]:
jsonlist = requests.get("https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&HISP=2&for=state:*").json()
states =[]
populations =[]
for list in jsonlist:
    states.append(list[0])
    populations.append(list[1])
data = {"State":states,"Population":populations
}
df=pd.DataFrame(data)
df = df.iloc[1:]
df["Population"]=pd.to_numeric(df["Population"])
df




,State,Population
1,Mississippi,100110
2,Missouri,268708
3,Montana,43289
4,Nebraska,219645
5,Nevada,900600
6,New Hampshire,54589
7,New Jersey,1856844
8,New Mexico,1032942
9,New York,3751058
10,North Carolina,1025830


In [38]:
alt.Chart(df).mark_area().encode(
    x="State",
    y="Population"
)

alt.Chart(...)

In [53]:
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')


alt.Chart(states).mark_geoshape().encode(
    color='Population:Q',
    tooltip=['State:N', 'Population:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'State', ['Population'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)


alt.Chart(...)